In [1]:
import torch
from transformers import (BertTokenizer,BertTokenizerFast, PreTrainedModel,
                          PreTrainedTokenizerBase)

from model import UIE
import coremltools as ct
import numpy as np
ckpt = '../save/uie_medium/model_best/'
tokenizer = BertTokenizer.from_pretrained(ckpt)
torch_model = UIE.from_pretrained(ckpt, torchscript=True)
torch_model.eval()

example_input = [torch.randint(10000, (5, ), dtype=torch.int32),
                 torch.tensor([0, 0, 0, 0, 0], dtype=torch.int32),
                 torch.tensor([1, 1, 1, 1, 1], dtype=torch.int32)]

traced_model = torch.jit.trace(torch_model, example_input)


Torch version 2.0.0 has not been tested with coremltools. You may run into unexpected errors. Torch 1.13.1 is the most recent version that has been tested.


In [2]:
mlmodel = ct.convert(
    traced_model,
    convert_to="mlprogram",
    inputs=[ct.TensorType(name="inputIds", shape=(ct.RangeDim(1, 128), ), dtype=np.int32),
            ct.TensorType(name="tokenTypeIds", shape=(ct.RangeDim(1, 128), ), dtype=np.int32),
            ct.TensorType(name="attentionMask", shape=(ct.RangeDim(1, 128), ), dtype=np.int32)],

)
mlmodel = ct.compression_utils.affine_quantize_weights(mlmodel)
spec = mlmodel.get_spec()
spec

Tuple detected at graph output. This will be flattened in the converted model.
Running MIL Common passes:   0%|          | 0/40 [00:00<?, ? passes/s]/Users/paulzhang/miniforge3/lib/python3.10/site-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:135: UserWarning: Output, '562', of the source model, has been renamed to 'var_562' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
/Users/paulzhang/miniforge3/lib/python3.10/site-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:135: UserWarning: Output, '568', of the source model, has been renamed to 'var_568' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Running MIL Clean up passes: 100%|██████████| 11/11 [00:00<00:00, 11.88 passes/s]


specificationVersion: 7
description {
  input {
    name: "inputIds"
    type {
      multiArrayType {
        shape: 1
        dataType: INT32
        shapeRange {
          sizeRanges {
            lowerBound: 1
            upperBound: 128
          }
        }
      }
    }
  }
  input {
    name: "tokenTypeIds"
    type {
      multiArrayType {
        shape: 1
        dataType: INT32
        shapeRange {
          sizeRanges {
            lowerBound: 1
            upperBound: 128
          }
        }
      }
    }
  }
  input {
    name: "attentionMask"
    type {
      multiArrayType {
        shape: 1
        dataType: INT32
        shapeRange {
          sizeRanges {
            lowerBound: 1
            upperBound: 128
          }
        }
      }
    }
  }
  output {
    name: "var_562"
    type {
      multiArrayType {
        dataType: FLOAT32
      }
    }
  }
  output {
    name: "var_568"
    type {
      multiArrayType {
        dataType: FLOAT32
      }
    }
  }
  m

In [3]:

ct.utils.rename_feature(spec, 'var_562', 'start_prob')
ct.utils.rename_feature(spec, 'var_568', 'end_prob')
mlmodel = ct.models.MLModel(spec, weights_dir=mlmodel.weights_dir)

mlmodel.save(f"{ckpt}UIE.mlpackage")


In [4]:
mlmodel.get_spec()

specificationVersion: 7
description {
  input {
    name: "inputIds"
    type {
      multiArrayType {
        shape: 1
        dataType: INT32
        shapeRange {
          sizeRanges {
            lowerBound: 1
            upperBound: 128
          }
        }
      }
    }
  }
  input {
    name: "tokenTypeIds"
    type {
      multiArrayType {
        shape: 1
        dataType: INT32
        shapeRange {
          sizeRanges {
            lowerBound: 1
            upperBound: 128
          }
        }
      }
    }
  }
  input {
    name: "attentionMask"
    type {
      multiArrayType {
        shape: 1
        dataType: INT32
        shapeRange {
          sizeRanges {
            lowerBound: 1
            upperBound: 128
          }
        }
      }
    }
  }
  output {
    name: "start_prob"
    type {
      multiArrayType {
        dataType: FLOAT32
      }
    }
  }
  output {
    name: "end_prob"
    type {
      multiArrayType {
        dataType: FLOAT32
      }
    }
  }

In [6]:
from uie_predictor import UIEPredictor
from pprint import pprint
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
ckpt = '../save/uie_medium/model_best/'
# schema = ['时间', '待办'] # Define the schema for entity extraction
# ie = UIEPredictor(model='uie-base',task_path=ckpt, schema=schema, split_sentence=False)
# pprint(ie("今天下午三点提醒我去买花,另外，明天下午三点叫我开会")) # Better print results using pprint


# schema = ['时间', '待办'] # Define the schema for entity extraction
# ie = UIEPredictor(model='uie-base',task_path=ckpt, engine="coreml", schema=schema, max_seq_len=64)
# pprint(ie("今天下午三点提醒我去买花。另外，明天两点到三点叫我去吃饭")) # Better print results using pprint

schema = ['todo', 'time'] # Define the schema for entity extraction
ie = UIEPredictor(model='uie-nano',task_path=ckpt, engine="pytorch", schema=schema, max_seq_len=64)
pprint(ie("每年进行一次公司文化宣传推广活动")) # Better print results using pprint

[2023-04-05 20:06:42,504] [    INFO] - >>> [CoreMLInferBackend] Creating Engine ...
[2023-04-05 20:06:43,580] [    INFO] - >>> [InferBackend] Engine Created ...


[{'todo': [{'end': 7,
            'probability': 0.9966165,
            'start': 0,
            'text': '提醒我去取快递'}]}]


In [ ]:
"""
无法识别数据
明天两点到三点叫我去吃饭
起床相关
睡觉相关
"""


In [7]:
import json
devs = []
with open("../data/uie_todo_data/dev.txt", "r", encoding='utf8') as f:
    lines = f.readlines()
    for line in lines:
        sentence = json.loads(line)
        if sentence["content"] not in devs:
            devs.append(sentence["content"])

In [8]:
for sentence in devs:
    schema = ['time', 'todo'] # Define the schema for entity extraction
    ie = UIEPredictor(model='uie-base',task_path=ckpt,engine="coreml", schema=schema, max_seq_len=64)
    pprint((sentence, ie(sentence))) # Better print results using pprint

[2023-04-05 20:06:59,286] [    INFO] - >>> [CoreMLInferBackend] Creating Engine ...
[2023-04-05 20:07:00,394] [    INFO] - >>> [InferBackend] Engine Created ...
[2023-04-05 20:07:00,552] [    INFO] - >>> [CoreMLInferBackend] Creating Engine ...


('明年去欧洲旅游',
 [{'time': [{'end': 2, 'probability': 0.9999956, 'start': 0, 'text': '明年'}],
   'todo': [{'end': 7,
             'probability': 0.9999944,
             'start': 2,
             'text': '去欧洲旅游'}]}])


[2023-04-05 20:07:01,785] [    INFO] - >>> [InferBackend] Engine Created ...
[2023-04-05 20:07:01,962] [    INFO] - >>> [CoreMLInferBackend] Creating Engine ...


('明天要去取快递',
 [{'time': [{'end': 2, 'probability': 0.9999949, 'start': 0, 'text': '明天'}],
   'todo': [{'end': 7,
             'probability': 0.99824375,
             'start': 3,
             'text': '去取快递'}]}])


[2023-04-05 20:07:02,997] [    INFO] - >>> [InferBackend] Engine Created ...
[2023-04-05 20:07:03,218] [    INFO] - >>> [CoreMLInferBackend] Creating Engine ...


...

('本月底要交水费和电费',
 [{'time': [{'end': 3, 'probability': 0.9999938, 'start': 0, 'text': '本月底'}],
   'todo': [{'end': 10,
             'probability': 0.9979803,
             'start': 4,
             'text': '交水费和电费'}]}])


[2023-04-05 20:09:43,829] [    INFO] - >>> [InferBackend] Engine Created ...
[2023-04-05 20:09:44,024] [    INFO] - >>> [CoreMLInferBackend] Creating Engine ...


('每天喝两杯水',
 [{'time': [{'end': 2, 'probability': 0.99984235, 'start': 0, 'text': '每天'}],
   'todo': [{'end': 6, 'probability': 0.9999613, 'start': 2, 'text': '喝两杯水'}]}])


[2023-04-05 20:09:45,061] [    INFO] - >>> [InferBackend] Engine Created ...


('下礼拜和朋友去温哥华游玩',
 [{'time': [{'end': 3, 'probability': 0.99999774, 'start': 0, 'text': '下礼拜'}],
   'todo': [{'end': 12,
             'probability': 0.9999742,
             'start': 6,
             'text': '去温哥华游玩'}]}])
